#Problem Statement
Prepare a prediction model for profit of 50_startups data.
Do transformations for getting better predictions of profit and
make a table containing R^2 value for each prepared model.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.stats.tests.test_influence
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
import math

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [17]:
data1=pd.read_csv("/content/50_Startups.csv")
data1=data1.copy()
data1.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [16]:
data1.describe()

,R&D Spend,Administration,Marketing Spend,Profit,State_California,State_Florida,State_New York
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,73721.615600,121344.639600,211025.097800,112012.639200,0.340000,0.320000,0.340000
std,45902.256482,28017.802755,122290.310726,40306.180338,0.478518,0.471212,0.478518
min,0.000000,51283.140000,0.000000,14681.400000,0.000000,0.000000,0.000000
25%,39936.370000,103730.875000,129300.132500,90138.902500,0.000000,0.000000,0.000000
50%,73051.080000,122699.795000,212716.240000,107978.190000,0.000000,0.000000,0.000000
75%,101602.800000,144842.180000,299469.085000,139765.977500,1.000000,1.000000,1.000000
max,165349.200000,182645.560000,471784.100000,192261.830000,1.000000,1.000000,1.000000


In [15]:
data1.corr()

,R&D Spend,Administration,Marketing Spend,Profit,State_California,State_Florida,State_New York
R&D Spend,1.000000,0.241955,0.724248,0.972900,-0.143165,0.105711,0.039068
Administration,0.241955,1.000000,-0.032154,0.200717,-0.015478,0.010493,0.005145
Marketing Spend,0.724248,-0.032154,1.000000,0.747766,-0.168875,0.205685,-0.033670
Profit,0.972900,0.200717,0.747766,1.000000,-0.145837,0.116244,0.031368
State_California,-0.143165,-0.015478,-0.168875,-0.145837,1.000000,-0.492366,-0.515152
State_Florida,0.105711,0.010493,0.205685,0.116244,-0.492366,1.000000,-0.492366
State_New York,0.039068,0.005145,-0.033670,0.031368,-0.515152,-0.492366,1.000000


In [18]:
data1=pd.get_dummies(data1,columns=['State'])
x = data1[['R&D Spend','Administration', 'Marketing Spend', 'State_California', 'State_Florida', 'State_New York']]
y = data1[['Profit']]

In [22]:
model1=sm.OLS(y,x).fit()
model1.predict()

array([192390.57136312, 189071.32010486, 182276.18673274, 173584.97618968,
       172277.13381658, 163473.80711834, 158099.29278416, 160155.64464935,
       151634.74332629, 154829.66252703, 135664.64259061, 135528.60078156,
       129282.91780691, 127431.24898633, 149694.38277549, 146143.63551376,
       116854.07452871, 130085.4099336 , 129149.72574272, 115594.18840736,
       116570.73444002, 117201.50508978, 114833.30513845, 110123.79610588,
       113294.37345155, 102200.26891872, 110765.30116884, 114279.80403012,
       101818.58738701, 101721.04202948,  99629.01053735,  97617.29632028,
        98988.2366031 ,  98061.35894841,  88974.70416303,  90420.00960612,
        75423.09286568,  89577.70222196,  69606.52160907,  83684.97603994,
        74762.74617655,  74956.31104912,  70575.99371388,  60100.26821881,
        64585.14721305,  47588.36471007,  56272.99268505,  46468.23200498,
        49123.07308293,  48185.03879271])

In [23]:
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Thu, 03 Feb 2022   Prob (F-statistic):           1.34e-27
Time:                        09:08:46   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
R&D Spend            0.8060      0.046     17.369      0.000       0.712       0.900
Administration      -0.0270      0.052     -0.517      0.608      -0.132       0.078
Marketing Spend      0.0270      0.017      1.574      0.123      -0.008       0.062
State_California  5.013e+04   6884.820      7.281      0.000    3.62e+04     6.4e+04
State_Florida     5.032e+04   7251.767      6.940      0.000    3.57e+04    6.49e+04
State_New York    5.008e+04   6952.587      7.204      0.000    3.61e+04    6.41e+04
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     2.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [25]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
vif["features"] = x.columns
vif.round(1)


,VIF Factor,features
0,2.5,R&D Spend
1,1.2,Administration
2,2.4,Marketing Spend
3,9.0,State_California
4,9.4,State_Florida
5,9.2,State_New York


Here, we can see VIF is greater than 5 ,we need to drop the states to improve the model.

In [27]:
new_x=data1[['R&D Spend','Administration', 'Marketing Spend']]

In [29]:
model2=sm.OLS(y,new_x).fit()
model2.predict()

array([202344.57734428, 202822.45833599, 176832.46925808, 174031.85688267,
       162066.48958189, 157209.89857957, 155379.52572476, 167853.35268393,
       160886.46954973, 149225.43959565, 128242.21035082, 122880.88787655,
       129643.63173806, 131211.08412208, 158493.44054532, 143916.06723873,
       117580.42905134, 138722.33960051, 127522.70751236, 112351.01509529,
       116604.47964216, 131302.93854504, 118286.48364369, 108184.96417459,
        99400.29339868, 103494.73671114, 112331.33287902, 122694.86020518,
       116983.21292402, 106054.00777744,  89893.26022013, 101182.37143623,
        91656.40675018,  91252.01167296, 102330.43877985,  77794.82783167,
        78743.37968362,  64637.13675208,  51359.67546453,  69257.33756751,
        73691.42945766,  61301.19383638,  60656.5642338 ,  55795.06874998,
        68980.25613213,  41558.8779388 ,  63313.52636367,  44378.68708525,
        17345.12345068,  42047.27494232])

In [30]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Profit   R-squared (uncentered):                   0.987
Model:                            OLS   Adj. R-squared (uncentered):              0.987
Method:                 Least Squares   F-statistic:                              1232.
Date:                Thu, 03 Feb 2022   Prob (F-statistic):                    1.17e-44
Time:                        11:11:03   Log-Likelihood:                         -545.82
No. Observations:                  50   AIC:                                      1098.
Df Residuals:                      47   BIC:                                      1103.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
R&D Spend           0.7180      0.065     11.047      0.000       0.587       0.849
Administration      0.3277      0.031     10.458      0.000       0.265       0.391
Marketing Spend     0.0822      0.022      3.733      0.001       0.038       0.126
==============================================================================
Omnibus:                        0.665   Durbin-Watson:                   1.361
Prob(Omnibus):                  0.717   Jarque-Bera (JB):                0.749
Skew:                          -0.126   Prob(JB):                        0.688
Kurtosis:                       2.456   Cond. No.                         9.76
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Squareroot transformation of X and Y**

In [33]:
x_sqrt=np.sqrt(new_x)
y_sqrt=np.sqrt(y)
model3=sm.OLS(y_sqrt,x_sqrt).fit()
model3.predict()


array([437.38702923, 443.12066262, 401.07858624, 406.23344227,
       383.4014534 , 382.76898864, 396.92804118, 410.31614476,
       404.50652027, 379.22655777, 358.54905794, 345.13892172,
       365.4506448 , 369.54149013, 404.61508616, 379.29684963,
       349.17869052, 380.2289465 , 358.20784852, 334.89340512,
       344.85838357, 373.04508369, 349.34526326, 331.29674295,
       320.77033576, 336.80722065, 349.39273959, 354.6111652 ,
       361.64201795, 342.54101508, 312.39150855, 335.3977204 ,
       315.92941776, 309.82454672, 335.82219327, 284.37839014,
       292.34623095, 250.11019862, 229.59617891, 270.69954588,
       283.97216265, 256.36560736, 257.1472518 , 248.74774792,
       276.81318568, 188.09594782, 225.8840976 , 176.14763786,
       120.55057691, 181.186506  ])

In [44]:
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Profit   R-squared (uncentered):                   0.994
Model:                            OLS   Adj. R-squared (uncentered):              0.993
Method:                 Least Squares   F-statistic:                              2396.
Date:                Thu, 03 Feb 2022   Prob (F-statistic):                    2.20e-51
Time:                        11:37:17   Log-Likelihood:                         -235.70
No. Observations:                  50   AIC:                                      477.4
Df Residuals:                      47   BIC:                                      483.1
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
R&D Spend           0.5012      0.054      9.277      0.000       0.393       0.610
Administration      0.4787      0.032     15.001      0.000       0.414       0.543
Marketing Spend     0.0822      0.030      2.699      0.010       0.021       0.143
==============================================================================
Omnibus:                        1.516   Durbin-Watson:                   1.524
Prob(Omnibus):                  0.469   Jarque-Bera (JB):                0.926
Skew:                           0.322   Prob(JB):                        0.629
Kurtosis:                       3.174   Cond. No.                         9.46
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We can note that R-square value of model 3 is greater than model 2,therefore we use model 3 for prediction .